In [0]:
create table sku 
(
sku_id int,
price_date date ,
price int
);
delete from sku;
insert into sku values 
(1,'2023-01-01',10)
,(1,'2023-02-15',15)
,(1,'2023-03-03',18)
,(1,'2023-03-27',15)
,(1,'2023-04-06',20);


Select *
from sku;

With tb as (Select sku_id ,price_date , 
dateadd(day ,-1, lead(price_date, 1,"2099-01-01") over(order by price_date)) as next , price
from sku ),

dates as (
Select min(price_date) as min_date , 
max(price_date) as max_date
from sku),

date_series as (
  select explode(
    sequence(
      (select min_date from dates),
      (select max_date from dates),
      interval 1 day
    )
  ) as min_date
)

Select min_date , price, 
price- lag(price ,1,10) over(order by min_date) as diff
from (
Select date_series.min_date , tb.price 
from date_series
join tb 
on date_series.min_date >= tb.price_date and date_series.min_date <= tb.next
where day(date_series.min_date) =1 ) as a